In [1]:
import sys,os
import argparse
import json
sys.path.append("/Users/tuhinsharma/Documents/sstech/projects/sia/models")
import pandas as pd
from pyspark.sql import SparkSession

from config import *
from model_platform.src.model.anomaly.profile_model.cluster_profile_model import \
    ClusterProfileModel
from model_platform.src.model.anomaly.profile_model.isolation_forest_profile_model import \
    IsolationForestProfileModel
from model_platform.src.model.anomaly.profile_model.oneclasssvm_profile_model import \
    OneClassSVMProfileModel
from utils.data_store.hdfs_data_store import HDFSDataStore
from utils.logging.pylogger import get_log_path, configure_logger
from model_platform.src.operationalization.anomaly.profile_anomaly.batch_score import *
from utils.logging.pylogger import get_log_path, configure_logger

{'is_nu': 14, 'is_ne': 5, 'is_uatone': 15, 'is_uafromne': 4, 'is_nwpu': 16, 'is_nwpp': 11, 'is_nactpu': 10, 'is_nactpp': 17, 'is_nacnpu': 19, 'is_nacnpp': 1}


In [2]:
data_source = "wgtraffic"
log_path = get_log_path(data_source, "profile_anomaly_batch_score")
logger = configure_logger(logger_name="profile_anomaly_batch_score", log_path=log_path, log_level=LOG_LEVEL)

logger.info(data_source)
app_name = data_source + "_profile_anomaly_batch_score"
spark = SparkSession.builder.appName(app_name).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel(PYSPARK_LOGLEVEL)

model_dict = load_profile_anomaly_model(spark=spark, data_source=data_source)
# logger.info(model_dict)

data_path = USER_PROFILE_DATA_PATH.format(data_source=data_source, entity_type=IP_ENTITY_TYPE,
                                      anomaly_type=PROFILE_ANOMALY_TYPE,
                                      time_window="hour")
data_store = HDFSDataStore(spark=spark)
input_sdf = data_store.read_spark_df_from_data_store(data_path=data_path, data_source=data_source,
                                                 entity_type=IP_ENTITY_TYPE,
                                                 anomaly_type=PROFILE_ANOMALY_TYPE,
                                                 time_window="hour")
input_sdf.show()
# logger.info(input_sdf.count())



2019-05-11 18:17:32.160 INFO <ipython-input-2-a51c85756cdf> - <module>: wgtraffic
+-------------+------+------+------+------+
|       src_ip|wgcat1|wgcat2|wgnum1|wgnum2|
+-------------+------+------+------+------+
|    54.85.1.3| False|  True|   242|   473|
|   54.85.5.15| False|  True|   297|   503|
|  54.85.10.16|  True| False|   235|   496|
|  54.85.20.40|  True| False|   235|   466|
|  54.85.23.10| False| False|   257|   422|
|  54.85.23.16| False|  True|   198|   463|
|    54.85.9.2| False| False|   219|   489|
|    54.85.9.5| False|  True|   222|   437|
|  54.85.12.37| False| False|   293|   473|
|  54.85.16.11| False| False|   251|   437|
|   54.85.17.2| False|  True|   233|   483|
|   54.85.17.9|  True| False|   215|   491|
|144.160.7.102|  True|  True|   231|   471|
|  54.85.21.33| False| False|   231|   535|
|  54.85.22.18|  True| False|   249|   448|
|   54.85.2.28|  True| False|   274|   443|
|   54.85.2.32|  True|  True|   256|   421|
|   54.85.9.20|  True| False|   209|  

In [12]:
input_sdf.sample(False, 0.01, seed=0).limit(100).show()

+------------+------+------+------+------+
|      src_ip|wgcat1|wgcat2|wgnum1|wgnum2|
+------------+------+------+------+------+
|54.85.115.17|  True|  True|   235|   482|
|  54.85.8.16|  True|  True|   280|   494|
|   54.85.4.8| False|  True|   274|   495|
| 54.85.16.21| False| False|   389|   368|
|  54.85.9.12| False|  True|   489|   366|
| 54.85.21.15| False| False|   517|   284|
|  54.85.5.13| False|  True|   456|   352|
|  54.85.7.29| False| False|   392|   290|
+------------+------+------+------+------+



In [3]:
time_window="hour"
entity_type=IP_ENTITY_TYPE
result_df_1 = model_dict[entity_type][PYSPARK_KMEANS_MODEL][time_window].score(
input_sdf)

In [4]:
model_isolation = model_dict[entity_type][SKLEARN_ISOLATION_FOREST_MODEL][time_window]
model_oneclasssvm = model_dict[entity_type][SKLEARN_ONECLASS_SVM_MODEL][time_window]
sc.broadcast(model_isolation)
sc.broadcast(model_oneclasssvm)




In [7]:
def score_map_isolation(x):
    df = pd.DataFrame([[x["wgcat1"],x["wgcat2"],x["wgnum1"],x["wgnum2"]]], columns=["wgcat1","wgcat2","wgnum1","wgnum2"])
    score = model_isolation.score(df)["PAS"][0]
    return score

def score_map_oneclasssvm(x):
    df = pd.DataFrame([[x["wgcat1"],x["wgcat2"],x["wgnum1"],x["wgnum2"]]], columns=["wgcat1","wgcat2","wgnum1","wgnum2"])
    score = model_oneclasssvm.score(df)["PAS"][0]
    return score
    

In [21]:
from time import time
t0 = time()
m = input_sdf.rdd.map(score_map_isolation).collect()

print(m)
time()-t0

[7.849460265884257, 21.386905604489392, 12.568020004764705, 8.694795797121492, 20.296140911154186, 30.123231130889046, 23.400646317966213, 22.039578924082925, 20.535450538796656, 13.256681880582017, 10.469565011840032, 23.86216490278606, 5.691277854489227, 41.65095213891433, 9.185235088616864, 14.649419263159956, 12.607385498731862, 28.573068406037798, 11.117137942678388, 6.513680318434417, 26.298196348820902, 19.889109380619647, 12.553602883530958, 0.10628787683329222, 16.443662442499438, 26.631306833621842, 13.600310452982251, 13.869954047672701, 4.591460535650887, 21.60722593435851, 25.705677709696726, 15.61179674403455, 6.812910039651875, 29.269168648479898, 24.593909601822375, 58.50114973382604, 0.9335804367368876, 12.764910342478434, 13.33397001264446, 17.389131054846388, 8.915943545437527, 18.165806142718473, 8.359048337929593, 18.239350501957425, 10.915710845796793, 13.431252418059616, 6.397689857614697, 4.2425068552329375, 26.178454978781115, 8.123755863333038, 8.5578216955785

297.9957900047302

In [18]:
from pyspark.sql.functions import concat, col, lit,struct

input_sdf = input_sdf.withColumn("hello",struct(col("wgcat1"), col("wgcat2"),col("wgnum1"),col("wgnum2")))

In [19]:
input_sdf.rdd.take(1)[0].hello["wgcat1"]

'False'

In [20]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import *

t0 = time()

# def score_map_isolation(x,centroids):
#     df = pd.DataFrame([[x["wgcat1"],x["wgcat2"],x["wgnum1"],x["wgnum2"]]], columns=["wgcat1","wgcat2","wgnum1","wgnum2"])

#     score = centroids.score(df)["PAS"][0]
#     return float(score)

# def udf_score_isolation(model_isolation):
#     return udf(lambda l: score_map_isolation(l, model_isolation),FloatType())

# result_score_sdf = input_sdf.withColumn("hi", udf_score_isolation("hello"))
# result_score_sdf.show(3)


def score_map_isolation(x):
    df = pd.DataFrame([[x["wgcat1"],x["wgcat2"],x["wgnum1"],x["wgnum2"]]], columns=["wgcat1","wgcat2","wgnum1","wgnum2"])

    score = model_isolation.score(df)["PAS"][0]
    return float(score)

squared_udf = udf(score_map_isolation, FloatType())
input_sdf.select("hello", squared_udf("hello").alias("id_squared")).show()

time()-t0

+--------------------+----------+
|               hello|id_squared|
+--------------------+----------+
|[False,True,242,473]|   7.84946|
|[False,True,297,503]| 21.386906|
|[True,False,235,496]|  12.56802|
|[True,False,235,466]|  8.694796|
|[False,False,257,...|  20.29614|
|[False,True,198,463]| 30.123232|
|[False,False,219,...| 23.400646|
|[False,True,222,437]|  22.03958|
|[False,False,293,...|  20.53545|
|[False,False,251,...| 13.256681|
|[False,True,233,483]| 10.469565|
|[True,False,215,491]| 23.862165|
| [True,True,231,471]|  5.691278|
|[False,False,231,...|  41.65095|
|[True,False,249,448]|  9.185235|
|[True,False,274,443]| 14.649419|
| [True,True,256,421]| 12.607386|
|[True,False,209,486]| 28.573069|
|[False,True,271,485]| 11.117138|
|[False,True,246,471]| 6.5136805|
+--------------------+----------+
only showing top 20 rows



287.5009779930115

In [ ]:

l = input_sdf.rdd.map(score_map_isolation)


In [ ]:
l.take(2)

In [ ]:

df["PAS_1"] = result_df_1["PAS"]
result_df_2 = model_dict[entity_type][SKLEARN_ISOLATION_FOREST_MODEL][
time_window].score(input_sdf)
df["PAS_2"] = result_df_2["PAS"]
result_df_3 = ].score(input_sdf)
df["PAS_3"] = result_df_3["PAS"]
df["PAS_" + data_source] = df.mean(axis=1)


In [ ]:
df